In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns # Retirar depois
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

In [ ]:
champions = pd.read_csv('data/championsdata.csv')
subchampions = pd.read_csv('data/runnerupsdata.csv')
total = pd.concat([champions, subchampions], ignore_index=True)
total.drop(["Win"], axis=1)
total

# 1. Preparar las variables (X) y el objetivo (y)
# ---------------------------------------------------------

# X = Todo MENOS 'Win'. 
# IMPORTANTE: También solemos quitar identificadores de texto como 'Team' 
# y a veces 'Year' o 'Game' para que el modelo se centre solo en estadísticas.
# Si dejas 'Team' (texto), los modelos matemáticos darán error.
X = total.drop(['Win', 'Team', 'Year', 'Game'], axis=1)

# y = SOLO la columna 'Win' (Ground Truth)
y = total['Win']

# 2. Dividir en Train y Test
# ---------------------------------------------------------
# test_size=0.2 significa que guardamos el 20% de los datos para el examen final
# random_state=42 sirve para que la división sea siempre igual cada vez que ejecutes (reproducibilidad)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

# Comprobación rápida de tamaños
print(f"Entrenamiento: {X_train.shape} (Filas, Columnas)")
print(f"Test: {X_test.shape} (Filas, Columnas)")

Entrenamiento: (352, 20) (Filas, Columnas)
Test: (88, 20) (Filas, Columnas)


In [ ]:
# LIMPIEZA VALORES NULOS

# TP - Tiros de campo de 3 puntos anotados
# TPA - Intentos de tres puntos
# TPP - porcentaje de tres puntos
#   TPP = TP / TPA
#   Si TPA == 0 entonces TPP es NaN

def get_percentage_nan_per_column(df):
  # retorna un df amb el % de NaNs de cada columna
  return df.isna().sum().sort_values()/len(df)*100.

percentage_nan = get_percentage_nan_per_column(total)
percentage_nan

total.fillna(0, inplace=True)

Year    0.000000
TOV     0.000000
BLK     0.000000
STL     0.000000
AST     0.000000
TRB     0.000000
DRB     0.000000
ORB     0.000000
FTP     0.000000
FTA     0.000000
FT      0.000000
PTS     0.000000
TPA     0.000000
TP      0.000000
FGP     0.000000
FGA     0.000000
FG      0.000000
MP      0.000000
Home    0.000000
Win     0.000000
Game    0.000000
Team    0.000000
PF      0.000000
TPP     2.045455
dtype: float64

MP   - Minutos jugados -> Cambiar por categoria OVERTIME (1 SI - 0 NO)
Game - Numero de partido de la serie -> INNECESARIO
Year - Año de la serie -> INNECESARIO
Team - Nombre del equipo -> ¿INNECESARIO? Analizar

# 1. La variable a predecir (Target)
Sin duda alguna, la variable objetivo es:

Win (0 o 1).

Es un problema clásico de clasificación binaria: queremos saber qué factores aumentan la probabilidad de que esa variable sea 1.

# 2. Las variables que más influyen (Hipótesis)
Si tuviera que apostar cuáles tendrán una correlación más fuerte con la victoria (excluyendo PTS, que es el resultado directo), diría que son estas cuatro, en este orden:

## 1. FGP (Porcentaje de Tiros de Campo) y TPP (Triples)
Es la métrica de eficiencia más pura. En la NBA, el equipo que tira con mejor porcentaje suele ganar.

Lógica: No importa cuántos tiros intentes (FGA), si no entran, no sumas. Un FGP alto suele ser el indicador #1 de victoria.

## 2. TOV (Pérdidas de Balón - Turnovers)
Esta debería tener una correlación negativa.

Lógica: Cada pérdida es una oportunidad de tiro que tu equipo pierde y (generalmente) una oportunidad fácil de contraataque para el rival. Quien cuida mejor el balón, suele ganar las finales.

## 3. Home (Factor Cancha)
En la NBA, y especialmente en las finales, jugar en casa es una ventaja estadística masiva.

Lógica: El público, el descanso, no viajar y el arbitraje (a veces) influyen. Espero ver una correlación positiva fuerte aquí.

## 4. DRB (Rebotes Defensivos)
Más que los ofensivos, los defensivos son clave.

Lógica: Si aseguras el rebote defensivo, evitas que el rival tenga "segundas oportunidades". "La defensa gana campeonatos", y el rebote es parte de la defensa.

⚠️ La "trampa" de los Puntos (PTS)
Técnicamente, PTS es la variable que más influye (si tienes más puntos que el rival, ganas el 100% de las veces). Pero en un modelo predictivo, a veces se excluye porque es una redundancia de la victoria.